In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import turtle
import random 
import time
from random import randrange
import mediapipe as mp
import cv2
import numpy as np
import time

delay=0.1
score=0
highestscore=0

#bodies
bodies=[]

#screen
s=turtle.Screen()
s.title("Snake game")
s.bgcolor("white")
s.setup(width=600, height=600)
s.addshape('A:\Files\PyPy\Snack_game\\head.gif')
s.addshape('A:\Files\PyPy\Snack_game\\body.gif')
s.addshape('A:\Files\PyPy\Snack_game\\food.gif')

#snack head
head=turtle.Turtle()
head.speed(0)

head.shape("A:\Files\PyPy\Snack_game\\head.gif")
head.color("grey")
head.fillcolor("blue")
head.penup()
head.goto(0,0)
head.direction="stop"

#food
food=turtle.Turtle()
food.speed(0)
food.shape("A:\Files\PyPy\Snack_game\\food.gif")
food.color("yellow")
food.fillcolor("green")
food.penup()
food.ht()
food.goto(0,200)
food.st()

#score board
sb=turtle.Turtle()
sb.shape("square")
sb.penup()
sb.ht()
sb.goto(140,280)
sb.write("Score:0  |  Heighest Score: 0")

def moveup():
    if head.direction!="down":
        head.direction="up"

def movedown():
    if head.direction!="up":
        head.direction="down"

def moveleft():
    if head.direction!="right":
        head.direction="left"

def moveright():
    if head.direction!="left":
        head.direction="right"

def movestop():
    head.direction="stop"

def move():
    if head.direction=="up":
        y=head.ycor()
        head.sety(y+20)
    if head.direction=="down":
        y=head.ycor()
        head.sety(y-20)
    if head.direction=="left":
        x=head.xcor()
        head.setx(x-20)
    if head.direction=="right":
        x=head.xcor()
        head.setx(x+20)

s.listen()
s.onkey(moveup, "Up")
s.onkey(movedown, "Down")
s.onkey(moveleft, "Left")
s.onkey(moveright, "Right")
s.onkey(movestop, "space")

thick = 4
prevx, prevy = 0,0
prevx1, prevy1 = 0,0

def index_raised(yi, y9):
    if (y9 - yi) > 40:
        return True

    return False

hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic
hand_landmark = hands.Hands(min_detection_confidence=0.6, min_tracking_confidence=0.6, max_num_hands=1)
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
draw = mp.solutions.drawing_utils

mask = np.ones((480, 640))*255
mask = mask.astype('uint8')

cap = cv2.VideoCapture(0)
sag = True

while True:
    try:
        s.update()
        #check collision with border
        if head.xcor()>300:
            head.setx(-300)
        if head.xcor()<-300:
            head.setx(300)
        if head.ycor()>300:
            head.sety(-300)
        if head.ycor()<-300:
            head.sety(300)

        #check collision with food
        if head.distance(food)<20:
            #move food to random location
            x=random.randint(-14,14)*20
            y=random.randint(-14,14)*20
            #x=randrange(-14,14)*20
            #y=randrange(-14,14)*20
            food.goto(x,y)

            #increase snack length
            body=turtle.Turtle()
            body.speed(0)
            body.penup()
            body.shape("A:\Files\PyPy\Snack_game\\body.gif")
            body.color("red")
            bodies.append(body)

            #update score 
            score+=10

            #increasing speed
            if delay>0:
                delay-=0.01

            if score>highestscore:
                highestscore=score
            sb.clear()
            sb.write("Score: {} | Highest Score: {}".format(score, highestscore))
        #move snake body
        for index in range(len(bodies)-1,0,-1):
            x=bodies[index-1].xcor()
            y=bodies[index-1].ycor()
            bodies[index].goto(x,y)

        if len(bodies)>0:
            x=head.xcor()
            y=head.ycor()
            bodies[0].goto(x,y)
        move()

        #check collision with self 
        for body in bodies:
            if body.distance(head)<20:
                time.sleep(1)
                head.goto(0,0)
                head.direction="stop"

                for body in bodies:
                    body.ht()
                bodies.clear()

                score=0
                delay=0.1

                sb.clear()
                sb.write("Score: {} Highest Score: {}".format(score, highestscore))
        if delay>0:
            time.sleep(delay)
        else:
            time.sleep(0.1)
        
        #mediapipe code starts here...
        _, frm = cap.read()
        wh = cv2.imread('white_new.png')
        frm = cv2.flip(frm, 1)

        rgb = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)

        op = hand_landmark.process(rgb)
        results = holistic.process(frm)
        
        cv2.line(frm, (212, 0), (212, 480), (0,255,0), 1)
        cv2.putText(frm, 'Left', (105, 240), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.line(frm, (424, 0), (424, 480), (0,255,0), 1)
        cv2.putText(frm, 'Right', (460, 240), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.line(frm, (0, 160), (640, 160), (0,255,0), 1)
        cv2.putText(frm, 'Up', (320, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.line(frm, (0, 320), (640, 320), (0,255,0), 1)
        cv2.putText(frm, 'Down', (320, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        
        if op.multi_hand_landmarks:
            for i in op.multi_hand_landmarks:
                draw.draw_landmarks(frm, i, hands.HAND_CONNECTIONS)

                #draw.draw_landmarks(frm, results.pose_landmarks, mp_holistic.FACE_CONNECTIONS)
                draw.draw_landmarks(wh, i, hands.HAND_CONNECTIONS)
                px, py = int(i.landmark[8].x*640), int(i.landmark[8].y*480)
                #print(x,y)
                
                if(212<px<424 and 0<py<160):
                    moveup()
                    test_text = cv2.putText(frm, 'Top', (px,py), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif(0<px<212 and 160<py<320):
                    moveleft()
                    test_text = cv2.putText(frm, 'left', (px,py), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif(212<px<424 and 320<py<480):
                    movedown()
                    test_text = cv2.putText(frm, 'Bottom', (px,py), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif(424<px<680 and 160<py<320):
                    moveright()
                    test_text = cv2.putText(frm, 'Right', (px,py), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif(212<px<424 and 160<py<320):
                    test_text = cv2.putText(frm, 'Hold', (px,py), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                else:
                    test_text = cv2.putText(frm, 'Invalid Move', (px,py), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        test = cv2.bitwise_or(wh, frm, mask=mask)
        wh[:, :, 1] = test[:, :, 1]
        wh[:, :, 2] = test[:, :, 2]

        cv2.imshow("paint app", frm)
        #cv2.imshow("whit bg", wh)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
 
    except Exception as e:
        print("something wrong")
        print(e)

        break
s.mainloop()


In [ ]:
cap.release()
cv2.destroyAllWindows()